# 高斯混合模型（Gaussian Mixture Model, GMM）详解

高斯混合模型是一种**基于概率模型的无监督聚类方法**，假设数据由多个高斯分布（正态分布）混合而成，并试图识别这些分布的参数。

它既可以用于聚类，也适用于密度估计、异常检测、图像分割等场景。

---

## 一、核心思想

- 假设数据由 $K$ 个高斯分布混合而成，每个分布具有自己的均值和协方差；
- 每个样本点由某个高斯分布“生成”，但我们不知道是哪个；
- GMM 用概率方式表达聚类结果（软聚类）；
- 每个点属于各个簇的概率由模型给出。

---

## 二、数学模型

### 1. 混合模型公式

数据点 $x$ 的概率密度为：

$$
p(x) = \sum_{k=1}^{K} \pi_k \cdot \mathcal{N}(x \mid \mu_k, \Sigma_k)
$$

其中：

- $\pi_k$：第 $k$ 个高斯成分的权重，满足 $\sum_k \pi_k = 1$
- $\mathcal{N}(x \mid \mu_k, \Sigma_k)$：均值为 $\mu_k$，协方差为 $\Sigma_k$ 的高斯分布

### 2. 多维高斯密度函数

$$
\mathcal{N}(x \mid \mu, \Sigma) = \frac{1}{(2\pi)^{d/2} |\Sigma|^{1/2}} \exp\left( -\frac{1}{2}(x - \mu)^T \Sigma^{-1} (x - \mu) \right)
$$

---

## 三、参数估计：EM算法（Expectation-Maximization）

由于我们不知道每个样本来自哪个高斯分布，使用 EM 算法进行参数估计。

### 步骤：

1. **初始化**：设定初始的 $\mu_k$, $\Sigma_k$, $\pi_k$
2. **E步（Expectation）**：计算每个样本来自每个簇的“责任”：

   $$
   \gamma_{ik} = \frac{\pi_k \cdot \mathcal{N}(x_i \mid \mu_k, \Sigma_k)}{\sum_{j=1}^K \pi_j \cdot \mathcal{N}(x_i \mid \mu_j, \Sigma_j)}
   $$

3. **M步（Maximization）**：更新参数：

   $$
   N_k = \sum_{i=1}^n \gamma_{ik}
   $$

   $$
   \mu_k = \frac{1}{N_k} \sum_{i=1}^n \gamma_{ik} x_i
   $$

   $$
   \Sigma_k = \frac{1}{N_k} \sum_{i=1}^n \gamma_{ik} (x_i - \mu_k)(x_i - \mu_k)^T
   $$

   $$
   \pi_k = \frac{N_k}{n}
   $$

4. **重复 E/M 步骤**直到收敛（对数似然函数收敛）。

---

## 四、GMM vs K-Means

| 特性               | GMM（软聚类）               | K-Means（硬聚类）         |
|--------------------|-----------------------------|----------------------------|
| 聚类边界           | 椭圆形/任意方向             | 圆形/等距划分              |
| 聚类概率           | 每个点有属于每个簇的概率    | 每个点只属于一个簇         |
| 对协方差建模       | 是                          | 否                         |
| 算法收敛           | EM算法（更复杂）            | Lloyd算法（更简单）        |
| 表现能力           | 更强                        | 一般                       |

---

## 五、优缺点

### ✅ 优点：
- 可建模椭圆形簇、不同大小和方向；
- 输出概率，更有解释性；
- 可用于密度估计、异常检测等。

### ❌ 缺点：
- 需要指定簇数 $K$；
- 对初始值敏感（可能收敛到局部最优）；
- 对离群点和高维数据敏感；
- 收敛速度比 K-Means 慢。

---

## 六、Python 示例（使用 scikit-learn）

```python
import numpy as np
from sklearn.mixture import GaussianMixture
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt

# 生成模拟数据
X, y_true = make_blobs(n_samples=300, centers=3, cluster_std=0.6, random_state=42)

# 拟合GMM
gmm = GaussianMixture(n_components=3, covariance_type='full', random_state=42)
gmm.fit(X)
labels = gmm.predict(X)

# 可视化结果
plt.scatter(X[:, 0], X[:, 1], c=labels, s=30, cmap='viridis')
plt.title("GMM Clustering Result")
plt.xlabel("Feature 1")
plt.ylabel("Feature 2")
plt.show()
```

---

## 七、选择簇数K的方法

1. **AIC (Akaike Information Criterion)**：
   - 越小越好；
   - 兼顾拟合度与模型复杂度。

2. **BIC (Bayesian Information Criterion)**：
   - 类似 AIC，但惩罚项更强；
   - 更偏好简单模型。

```python
aic_vals = []
bic_vals = []
ks = range(1, 10)

for k in ks:
    gmm = GaussianMixture(n_components=k, random_state=42)
    gmm.fit(X)
    aic_vals.append(gmm.aic(X))
    bic_vals.append(gmm.bic(X))

plt.plot(ks, aic_vals, label='AIC')
plt.plot(ks, bic_vals, label='BIC')
plt.xlabel("Number of Components (K)")
plt.ylabel("Score")
plt.legend()
plt.show()
```

---

## 八、总结

高斯混合模型是一种强大的概率聚类方法，能处理复杂形状的数据分布，并提供每个点的簇分布概率。虽然对初始值较敏感，但在许多实际应用中效果优异，尤其适合软聚类场景。

---

In [13]:
import numpy as np
import matplotlib.pyplot as plt

In [14]:
np.random.seed(0)
mu_m = 1.71
sigma_m = 0.056
num_m = 100000
rand_data_m = np.random.normal(mu_m, sigma_m, num_m)
y_m = np.ones(num_m)

In [15]:
np.random.seed(0)
nu_w = 1.58
sigma_w = 0.051
num_w = 100000
rand_data_w = np.random.normal(nu_w, sigma_w, num_w)
y_w = np.zeros(num_w)
rand_data_w

array([1.66996667, 1.60040802, 1.62991564, ..., 1.61703528, 1.52044064,
       1.51445441])

In [16]:
data = np.append(rand_data_m, rand_data_w)
data = data.reshape(-1, 1)
y = np.append(y_m, y_w)
print(data)
print(y)

[[1.80878693]
 [1.7324088 ]
 [1.76480933]
 ...
 [1.61703528]
 [1.52044064]
 [1.51445441]]
[1. 1. 1. ... 0. 0. 0.]


In [18]:
from sklearn.mixture import GaussianMixture
g = GaussianMixture(n_components=2, random_state=0,tol=1e-3,max_iter=1000)
g.fit(data)
print(g.means_)
print(g.covariances_)
print(g.weights_)

[[1.58168229]
 [1.71356861]]
[[[0.00261101]]

 [[0.00287461]]]
[0.51926735 0.48073265]


In [19]:
from sklearn.metrics import accuracy_score
y_hat = g.predict(data)
print(accuracy_score(y, y_hat))

0.888835
